In [2]:
!pip install nolds
!pip install hmmlearn
!pip install yfinance
!pip install statsmodels
!pip install hurst ruptures

python(65022) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


python(65023) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


python(65024) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


python(65025) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


python(65026) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [3]:
import yfinance as yf
import datetime
from numpy import *
from pylab import plot, show

import pandas as pd
import numpy as np

import numpy as np
from pathlib import Path
from hmmlearn.hmm import GaussianHMM
from sklearn.preprocessing import StandardScaler
#from google.colab import drive

In [4]:
#Umcomment this method when running from google colab
#def load_data_from_drive(start_date, end_date):
  #drive.mount('/content/drive', force_remount=True)
  #base_path = Path("/content/drive/My Drive/WQU/Capstone/data/parquet_out")
  #dfs = []
  #for d in sorted(base_path.iterdir()):
      #if d.is_dir() and start_date <= d.name <= end_date:
          #dfs.append(pd.read_parquet(d))

  #return pd.concat(dfs, ignore_index=True)

def load_data(start_date, end_date):
  base_path = Path("./data/parquet_out")
  dfs = []
  for d in sorted(base_path.iterdir()):
      if d.is_dir() and start_date <= d.name <= end_date:
          dfs.append(pd.read_parquet(d))

  return pd.concat(dfs, ignore_index=True)




In [5]:
df = load_data("2026-01-01", "2026-01-12")
print(df.head())
print(df.tail())

                        time    symbol bidqty askqty       ltp   atp   ask  \
0  2026-01-01T09:15:00+05:30  NIFTY 50      0      0  26183.50  0.00  0.00   
1  2026-01-01T09:16:00+05:30  NIFTY 50      0      0  26194.30  0.00  0.00   
2  2026-01-01T09:17:00+05:30  NIFTY 50      0      0  26189.55  0.00  0.00   
3  2026-01-01T09:18:00+05:30  NIFTY 50      0      0  26179.50  0.00  0.00   
4  2026-01-01T09:19:00+05:30  NIFTY 50      0      0  26167.80  0.00  0.00   

    bid oi tot_vol tot_buyqty tot_sellqty ltq                  ltt level2  \
0  0.00  0       0          0           0   0  2026-01-01 09:15:00   None   
1  0.00  0       0          0           0   0  2026-01-01 09:16:00   None   
2  0.00  0       0          0           0   0  2026-01-01 09:17:00   None   
3  0.00  0       0          0           0   0  2026-01-01 09:18:00   None   
4  0.00  0       0          0           0   0  2026-01-01 09:19:00   None   

  recv_time bcast_time security_id  
0         0       None      256

**Function to get 1 min data**

In [8]:
def fill_intraday_minutes(day_df):
    day = day_df.index[0].date()
    full_index = pd.date_range(
        start=pd.Timestamp(f"{day} 09:15"),
        end=pd.Timestamp(f"{day} 15:30"),
        freq="1min"
    )
    return day_df.reindex(full_index).ffill()

def get_data_one_min_freq(start_date, end_date):
  df = load_data(start_date=start_date, end_date=end_date)
  df.drop(columns=['time', 'symbol', 'security_id', 'level2', 'bcast_time'], inplace=True)
  df['ltt'] = pd.to_datetime(df['ltt'])
  df = df.sort_values('ltt').set_index('ltt')
  df_1min = (
    df
    .groupby(df.index.date, group_keys=False)
    .apply(fill_intraday_minutes)
  )
  return df_1min



In [13]:
df_1min = get_data_one_min_freq("2026-01-01", "2026-01-12")
print(df_1min.index.to_series().diff().value_counts())
print(df_1min.groupby(df_1min.index.date).size().value_counts())

0 days 00:01:00    3000
0 days 17:45:00       5
2 days 17:45:00       2
Name: count, dtype: int64
376    8
Name: count, dtype: int64


**Function to get 5 mins data**

In [16]:
def fill_intraday_5min(day_df):
    day = day_df.index[0].date()

    # Create fixed 5-min intraday grid
    full_index = pd.date_range(
        start=pd.Timestamp(f"{day} 09:15"),
        end=pd.Timestamp(f"{day} 15:30"),
        freq="5min"
    )

    # Resample to 5-min bars (last price per bar)
    day_5min = (
        day_df
        .resample("5min")
        .last()
    )

    # Reindex to full grid and forward-fill within day
    return day_5min.reindex(full_index).ffill()


def get_data_5min_freq(start_date, end_date):
    df = load_data(start_date=start_date, end_date=end_date)

    df.drop(
        columns=['time', 'symbol', 'security_id', 'level2', 'bcast_time'],
        inplace=True
    )

    df['ltt'] = pd.to_datetime(df['ltt'])
    df = df.sort_values('ltt').set_index('ltt')

    df = df[~df.index.isna()]

    df_5min = (
        df
        .groupby(df.index.date, group_keys=False)
        .apply(fill_intraday_5min)
    )

    return df_5min


In [18]:
df_5min = get_data_5min_freq("2026-01-01", "2026-01-12")
print(df_5min.index.to_series().diff().value_counts())
print(df_5min.groupby(df_5min.index.date).size().value_counts())

0 days 00:05:00    600
0 days 17:45:00      5
2 days 17:45:00      2
Name: count, dtype: int64
76    8
Name: count, dtype: int64


In [20]:
df_5min.groupby(df_5min.index.date).size().head()

2026-01-01    76
2026-01-02    76
2026-01-05    76
2026-01-06    76
2026-01-07    76
dtype: int64

**Function to get 15 mins data**

In [23]:
def fill_intraday_15min(day_df):
    day = day_df.index[0].date()

    # Create fixed 15-min intraday grid
    full_index = pd.date_range(
        start=pd.Timestamp(f"{day} 09:15"),
        end=pd.Timestamp(f"{day} 15:30"),
        freq="15min"
    )

    # Resample to 5-min bars (last price per bar)
    day_15min = (
        day_df
        .resample("15min")
        .last()
    )

    # Reindex to full grid and forward-fill within day
    return day_15min.reindex(full_index).ffill()


def get_data_15min_freq(start_date, end_date):
    df = load_data(start_date=start_date, end_date=end_date)

    df.drop(
        columns=['time', 'symbol', 'security_id', 'level2', 'bcast_time'],
        inplace=True
    )

    df['ltt'] = pd.to_datetime(df['ltt'])
    df = df.sort_values('ltt').set_index('ltt')

    df = df[~df.index.isna()]

    df_15min = (
        df
        .groupby(df.index.date, group_keys=False)
        .apply(fill_intraday_15min)
    )

    return df_15min


In [25]:
df_15min = get_data_15min_freq("2026-01-01", "2026-01-12")
print(df_15min.index.to_series().diff().value_counts())
print(df_15min.groupby(df_15min.index.date).size().value_counts())

0 days 00:15:00    200
0 days 17:45:00      5
2 days 17:45:00      2
Name: count, dtype: int64
26    8
Name: count, dtype: int64


In [27]:
df_15min.tail()

,bidqty,askqty,ltp,atp,ask,bid,oi,tot_vol,tot_buyqty,tot_sellqty,ltq,recv_time
2026-01-12 14:30:00,0,0,25760.80,0.00,0.00,0.00,0,0,0,0,0,0
2026-01-12 14:45:00,0,0,25810.45,0.00,0.00,0.00,0,0,0,0,0,0
2026-01-12 15:00:00,0,0,25778.50,0.00,0.00,0.00,0,0,0,0,0,0
2026-01-12 15:15:00,0,0,25806.10,0.00,0.00,0.00,0,0,0,0,0,0
2026-01-12 15:30:00,0,0,25806.10,0.00,0.00,0.00,0,0,0,0,0,0


In [29]:
df_15min.groupby(df_15min.index.date).size().head()

2026-01-01    26
2026-01-02    26
2026-01-05    26
2026-01-06    26
2026-01-07    26
dtype: int64

Function to get 60 mins data

In [32]:
NUMERIC_COLS = [
    "tot_vol", "ask", "bid", "oi", "askqty", "bidqty",
    "tot_buyqty", "tot_sellqty", "ltq", "atp"
]

for col in NUMERIC_COLS:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")



PRICE_COL = "ltp"
VOLUME_COL = "tot_vol"

LAST_COLS = [
    "ask", "bid", "oi", "askqty", "bidqty",
    "tot_buyqty", "tot_sellqty", "ltq", "atp", "recv_time"
]



def build_agg_dict(df):
  agg = {
      PRICE_COL: ["first", "max", "min", "last"],
      VOLUME_COL: "sum"
  }

  for col in LAST_COLS:
    if col in df.columns:
      agg[col] = "last"
  return agg


def get_data_60min_freq(start_date, end_date):
    df = load_data(start_date=start_date, end_date=end_date)

    df.drop(
        columns=['time', 'symbol', 'security_id', 'level2', 'bcast_time'],
        inplace=True
    )


    NUMERIC_COLS = [
        "tot_vol", "ask", "bid", "oi",
        "askqty", "bidqty", "tot_buyqty",
        "tot_sellqty", "ltq", "atp"
    ]

    for col in NUMERIC_COLS:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")

    df['ltt'] = pd.to_datetime(df['ltt'], errors="coerce")
    df = df.sort_values('ltt').set_index('ltt')
    df = df[~df.index.isna()]

    df_60min = (
        df
        .groupby(df.index.date, group_keys=False)
        .apply(fill_intraday_60min)
    )

    return df_60min


def fill_intraday_60min(day_df):
    day = day_df.index[0].date()

    full_index = pd.date_range(
        start=pd.Timestamp(f"{day} 09:15"),
        end=pd.Timestamp(f"{day} 15:15"),
        freq="60min"
    )

    agg_dict = build_agg_dict(day_df)

    day_60min = (
    day_df
    .resample(
        "60min",
        origin=pd.Timestamp(f"{day} 09:15"),
        label="right",
        closed="right"
    )
    .agg(agg_dict)
    )

    day_60min.columns = [
        f"{c[0]}_{c[1]}" if isinstance(c, tuple) else c
        for c in day_60min.columns
    ]

    return day_60min.reindex(full_index).ffill()



In [34]:
df_60min = get_data_60min_freq("2026-01-01", "2026-01-12")
print(df_60min.index.to_series().diff().value_counts())
print(df_60min.groupby(df_60min.index.date).size().value_counts())

0 days 01:00:00    48
0 days 18:00:00     5
2 days 18:00:00     2
Name: count, dtype: int64
7    8
Name: count, dtype: int64


In [36]:
df_60min.index.name = 'time'
df_60min.tail()

,ltp_first,ltp_max,ltp_min,ltp_last,tot_vol_sum,ask_last,bid_last,oi_last,askqty_last,bidqty_last,tot_buyqty_last,tot_sellqty_last,ltq_last,atp_last,recv_time_last
time,,,,,,,,,,,,,,,
2026-01-12 11:15:00,25550.40,25595.30,25528.80,25528.80,0,0.0,0.0,0,0,0,0,0,0,0.0,0
2026-01-12 12:15:00,25526.15,25578.30,25475.00,25577.45,0,0.0,0.0,0,0,0,0,0,0,0.0,0
2026-01-12 13:15:00,25583.40,25732.20,25583.40,25725.15,0,0.0,0.0,0,0,0,0,0,0,0.0,0
2026-01-12 14:15:00,25726.60,25751.55,25689.05,25729.15,0,0.0,0.0,0,0,0,0,0,0,0.0,0
2026-01-12 15:15:00,25734.05,25812.45,25705.70,25784.05,0,0.0,0.0,0,0,0,0,0,0,0.0,0


In [40]:
# Generate Returns 


In [42]:
# ===============================================================
#  MARKET-DYNAMICS  –  Diagnostics  (quiet & threshold-tunable)
# ===============================================================
import numpy as np, pandas as pd, warnings
from statsmodels.tsa.stattools    import adfuller, kpss
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tools.sm_exceptions import InterpolationWarning
from scipy.stats  import kurtosis
from hurst        import compute_Hc
import ruptures as rpt

# ── silence spammy warnings ─────────────────────────────────────
warnings.filterwarnings("ignore", category=InterpolationWarning)
warnings.filterwarnings("ignore", category=UserWarning,
                        module="statsmodels.tsa.stattools")
np.seterr(divide="ignore", invalid="ignore")

# ── load price series (ltp) ────────────────────────────────────
df_1m = get_data_one_min_freq("2016-01-01", "2025-12-31")
if "time" in df_1m.columns:
    df_1m = df_1m.set_index("time")
df_1m.index = (df_1m.index.tz_localize("Asia/Kolkata")
               if df_1m.index.tz is None
               else df_1m.index.tz_convert("Asia/Kolkata"))
close   = pd.to_numeric(df_1m["ltp"], errors="coerce").dropna()
returns = close.pct_change().dropna()

# ── helpers ────────────────────────────────────────────────────
WINDOW = 250
roll_p = lambda s, fn: s.rolling(WINDOW).apply(lambda x: fn(x)[1], raw=False)

adf_p  = roll_p(returns, lambda x: adfuller(x, maxlag=12))
kpss_p = roll_p(returns, lambda x: kpss(x, nlags="auto"))
lb_df  = acorr_ljungbox(returns**2, lags=[10, 20], return_df=True)

kurt_ex  = returns.rolling(WINDOW).apply(
    lambda x: kurtosis(x, fisher=True, bias=False), raw=False
)
tail = returns.abs().sort_values(ascending=False)
k    = max(int(0.05 * len(tail)), 1)
hill_alpha = k / np.log(tail.iloc[:k] / tail.iloc[k-1]).sum()

rv  = returns.rolling(30).std().dropna()
H, *_ = compute_Hc(rv, kind="change", simplified=True)
n_breaks = len(rpt.Binseg("l2").fit(returns.values).predict(pen=10)) - 1

# ── summary ────────────────────────────────────────────────────
summary = pd.Series({
    "ADF pass-fraction"     : (adf_p  < 0.05).mean(),
    "KPSS pass-fraction"    : (kpss_p > 0.05).mean(),
    "LB p-lag10"            : lb_df.loc[10, "lb_pvalue"],
    "LB p-lag20"            : lb_df.loc[20, "lb_pvalue"],
    "Kurtosis>2 fraction"   : (kurt_ex > 2).mean(),
    "Hill alpha"            : hill_alpha,
    "Hurst (rv)"            : H,
    "Breakpoints"           : n_breaks
})
display(summary)

# ── gate rules (tweakable) ─────────────────────────────────────
LB_THRESHOLD   = 0.10   # raise from 0.05 if clustering is weak
HURST_MIN      = 0.55

gate_pass = (
    summary["ADF pass-fraction"]   >= 0.70 and
    summary["KPSS pass-fraction"]  >= 0.70 and
    summary["LB p-lag10"] < LB_THRESHOLD and
    summary["LB p-lag20"] < LB_THRESHOLD and
    (summary["Kurtosis>2 fraction"] > 0.30 or summary["Hill alpha"] < 4) and
    summary["Hurst (rv)"] > HURST_MIN
)

print("\nMARKET-DYNAMICS  GATE:", "PASS ✅" if gate_pass else "FAIL ❌")


ADF pass-fraction      0.993239
KPSS pass-fraction     0.957173
LB p-lag10             0.984842
LB p-lag20             0.999973
Kurtosis>2 fraction    0.725304
Hill alpha             2.981839
Hurst (rv)             0.679603
Breakpoints            0.000000
dtype: float64


MARKET-DYNAMICS  GATE: FAIL ❌


In [ ]:
# PATH A 

In [ ]:
# ────────────────────────────────────────────────────────
#  PATH A  –  Lag Sweep on 1-min Returns
#      Goal: find the lags where volatility clustering (ARCH-type)
#            is statistically significant.
# ────────────────────────────────────────────────────────
import pandas as pd
from statsmodels.stats.diagnostic import acorr_ljungbox

# 1-min absolute-squared returns
lags  = [1, 2, 3, 5, 10, 15, 20, 30, 60]       # minutes
lb_df = acorr_ljungbox(returns**2, lags=lags, return_df=True)

# Assemble p-values in a tidy DataFrame
lag_sweep = pd.DataFrame({
    "Lag (min)" : lags,
    "LB p-value": lb_df["lb_pvalue"].values
})
display(lag_sweep)

# Highlight the first lag that shows clustering
first_sig = lag_sweep.loc[lag_sweep["LB p-value"] < 0.05]
if not first_sig.empty:
    print(f"\n✅ Volatility clustering detected at lag {first_sig.iloc[0,0]}-min "
          f"(p = {first_sig.iloc[0,1]:.4f})")
else:
    print("\nℹ️  No Ljung-Box p-value below 0.05 for lags 1–60 min")


In [ ]:
# PATH B 

In [ ]:
# ─────────────────────────────────────────────────────────────
#  PATH B – Horizon Sweep (5-min & 15-min returns)
#      Goal: see whether volatility clustering appears after
#            smoothing 1-min noise.
# ─────────────────────────────────────────────────────────────
from statsmodels.stats.diagnostic import acorr_ljungbox
import pandas as pd

horizons   = {"5min": [1, 2, 4, 8],   # lags in multiples of the horizon
              "15min": [1, 2, 3, 4]}  # (e.g., 4 × 15 min = 1 hour)

results = []

for tf, lags in horizons.items():
    # resample close prices to tf bars and compute returns
    r_tf = close.resample(tf).last().pct_change().dropna()
    
    lb_df = acorr_ljungbox(r_tf**2, lags=lags, return_df=True, model_df=0)
    
    res = pd.DataFrame({
        "Horizon"   : tf,
        "Lag(mult)" : lags,
        "LB p-val"  : lb_df["lb_pvalue"].values
    })
    results.append(res)

lb_sweep_tf = pd.concat(results, ignore_index=True)
display(lb_sweep_tf)

# Flag first significant clustering per horizon
for tf in lb_sweep_tf["Horizon"].unique():
    sub = lb_sweep_tf[lb_sweep_tf["Horizon"] == tf]
    sig = sub[sub["LB p-val"] < 0.05]
    if not sig.empty:
        lag = sig.iloc[0]["Lag(mult)"]
        p   = sig.iloc[0]["LB p-val"]
        print(f"✅ {tf}: clustering at lag {lag}×{tf} (p = {p:.4f})")
    else:
        print(f"ℹ️  {tf}: no significant clustering (p < 0.05) up to max lag tested")


In [ ]:
# Market-Dynamics Gate” cell—updated for 5-minute clustering (Path B)

In [ ]:

import numpy as np, pandas as pd, warnings
from statsmodels.tsa.stattools    import adfuller, kpss
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tools.sm_exceptions import InterpolationWarning
from scipy.stats  import kurtosis
from hurst        import compute_Hc
import ruptures as rpt

# ── Silence warning spam ──────────────────────────────────────
warnings.filterwarnings("ignore", category=InterpolationWarning)
warnings.filterwarnings("ignore", category=UserWarning,
                        module="statsmodels.tsa.stattools")
np.seterr(divide="ignore", invalid="ignore")

# ── 1) Load 1-minute price series (ltp) ───────────────────────
df_1m = get_data_one_min_freq("2016-01-01", "2025-12-31")
if "time" in df_1m.columns:
    df_1m = df_1m.set_index("time")
df_1m.index = (df_1m.index.tz_localize("Asia/Kolkata")
               if df_1m.index.tz is None
               else df_1m.index.tz_convert("Asia/Kolkata"))
close   = pd.to_numeric(df_1m["ltp"], errors="coerce").dropna().astype(float)
ret_1m  = close.pct_change().dropna()

# ── 2) 5-minute returns for clustering check ─────────────────
ret_5m  = close.resample("5min").last().pct_change().dropna()
lb5_p   = acorr_ljungbox(ret_5m**2, lags=[1], return_df=True)["lb_pvalue"].iloc[0]

# ── 3) Stationarity on 1-min returns ─────────────────────────
WINDOW = 250
adf_p  = ret_1m.rolling(WINDOW).apply(lambda x: adfuller(x, maxlag=12)[1], raw=False)
kpss_p = ret_1m.rolling(WINDOW).apply(lambda x: kpss(x, nlags="auto")[1], raw=False)

# ── 4) Heavy tails (1-min) ───────────────────────────────────
kurt_ex   = ret_1m.rolling(WINDOW).apply(
    lambda x: kurtosis(x, fisher=True, bias=False), raw=False
)
tail       = ret_1m.abs().sort_values(ascending=False)
k          = max(int(0.05 * len(tail)), 1)
hill_alpha = k / np.log(tail.iloc[:k] / tail.iloc[k-1]).sum()

# ── 5) Vol-persistence (Hurst of 5-min RV) ───────────────────
rv_5m = ret_5m.rolling(12).std().dropna()   # 1-hour window
H, *_ = compute_Hc(rv_5m, kind="change", simplified=True)

# ── 6) Structural breaks (1-min) ─────────────────────────────
n_breaks = len(rpt.Binseg("l2").fit(ret_1m.values).predict(pen=10)) - 1

# ── 7) Summary table ─────────────────────────────────────────
summary = pd.Series({
    "ADF pass-fraction"     : (adf_p  < 0.05).mean(),
    "KPSS pass-fraction"    : (kpss_p > 0.05).mean(),
    "LB p-val 5-min lag-1"  : lb5_p,
    "Kurtosis>2 fraction"   : (kurt_ex > 2).mean(),
    "Hill alpha"            : hill_alpha,
    "Hurst (rv-5m)"         : H,
    "Breakpoints"           : n_breaks
})
display(summary)

# ── 8) Gate criteria ─────────────────────────────────────────
gate_pass = (
    summary["ADF pass-fraction"]  >= 0.70    and
    summary["KPSS pass-fraction"] >= 0.70    and
    summary["LB p-val 5-min lag-1"] < 0.05   and  # new clustering rule
    (summary["Kurtosis>2 fraction"] > 0.30 or summary["Hill alpha"] < 4) and
    summary["Hurst (rv-5m)"] > 0.55
)

print("\nMARKET-DYNAMICS  GATE:", "PASS ✅" if gate_pass else "FAIL ❌")
